In [334]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

# mpl.rcParams['figure.figsize'] = (8, 6)
# mpl.rcParams['axes.grid'] = False


Plan:
 - Load sets
 - Clean:
    - remove duplicate entries from sets
    - Drop the unimportant coloumns that we found
    - Make querters into hours
    - remove values from train_targets that are nan in both x and y sets
    - make time features
    - drop datetime coloumns
 - Set prep:
    - 75/25 fordeling. 75% av OBSERVED er trening, resterende 25% er validation, 75% av ESTIMATED er trening, resterende 25% er validation
- Concat the different locations into 1 set, where location is a feature as well
   - 1 hot encoding
- Make a time series model

#### Loading all datasets

In [335]:
#A
y_a = pd.read_parquet('../data/A/train_targets.parquet')
X_test_estimated_a = pd.read_parquet('../data/A/X_test_estimated.parquet')
X_train_estimated_a = pd.read_parquet('../data/A/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('../data/A/X_train_observed.parquet')

# B
y_b = pd.read_parquet('../data/B/train_targets.parquet')
X_test_estimated_b = pd.read_parquet('../data/B/X_test_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../data/B/X_train_estimated.parquet')
X_train_observed_b = pd.read_parquet('../data/B/X_train_observed.parquet')

# C
y_c = pd.read_parquet('../data/C/train_targets.parquet')
X_test_estimated_c = pd.read_parquet('../data/C/X_test_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../data/C/X_train_estimated.parquet')
X_train_observed_c = pd.read_parquet('../data/C/X_train_observed.parquet')

#### Removing duplicate entries from the sets if any exists

In [336]:
# Function that removes the duplicates if it finds duplicates in the specified coloumn
def remove_duplicates_in_coloumn(df, col):
    duplicate_mask = df[col].duplicated(keep="first")
    if duplicate_mask.any():
        df = df[~duplicate_mask]
    return df

In [337]:
#A
y_a = remove_duplicates_in_coloumn(y_a, "time")
X_test_estimated_a = remove_duplicates_in_coloumn(X_test_estimated_a, "date_forecast")
X_train_estimated_a = remove_duplicates_in_coloumn(X_train_estimated_a, "date_forecast")
X_train_observed_a = remove_duplicates_in_coloumn(X_train_observed_a, "date_forecast")

#B
y_b = remove_duplicates_in_coloumn(y_b, "time")
X_test_estimated_b = remove_duplicates_in_coloumn(X_test_estimated_b, "date_forecast")
X_train_estimated_b = remove_duplicates_in_coloumn(X_train_estimated_b, "date_forecast")
X_train_observed_b = remove_duplicates_in_coloumn(X_train_observed_b, "date_forecast")

#C
y_c = remove_duplicates_in_coloumn(y_c, "time")
X_test_estimated_c = remove_duplicates_in_coloumn(X_test_estimated_c, "date_forecast")
X_train_estimated_c = remove_duplicates_in_coloumn(X_train_estimated_c, "date_forecast")
X_train_observed_c = remove_duplicates_in_coloumn(X_train_observed_c, "date_forecast")


In [338]:
#list of all estimated and observed sets

list_of_all_estimated_and_observed_sets = [X_test_estimated_a, X_train_estimated_a, X_train_observed_a,
                                           X_test_estimated_b, X_train_estimated_b, X_train_observed_b,
                                           X_test_estimated_c, X_train_estimated_c, X_train_observed_c]

#### Dropping some coloumns for the bants

In [339]:
for set in list_of_all_estimated_and_observed_sets:
    # set.drop("snow_density:kgm3", axis=1, inplace=True)
    # these 2 had a lot of NaN values
    # set.drop("ceiling_height_agl:m", axis=1, inplace=True) 
    # set.drop("cloud_base_agl:m", axis=1,inplace=True) # could potentially not drop this, but set all nan values to 0
    set.drop("snow_density:kgm3", axis=1, inplace=True)

#### Converting degree features to vector

In [340]:
def convert_from_degree_to_ciruclar(df, feature):
    df[feature+'_sin'] = np.sin(np.radians(df[feature]))
    df[feature+'_cos'] = np.cos(np.radians(df[feature]))
    df = df.drop(feature, axis = 1)
    return df

In [341]:
X_train_estimated_a = convert_from_degree_to_ciruclar(X_train_estimated_a, "sun_azimuth:d")
X_train_observed_a = convert_from_degree_to_ciruclar(X_train_observed_a, "sun_azimuth:d")
X_test_estimated_a = convert_from_degree_to_ciruclar(X_test_estimated_a, "sun_azimuth:d")
X_train_estimated_b = convert_from_degree_to_ciruclar(X_train_estimated_b, "sun_azimuth:d")
X_train_observed_b = convert_from_degree_to_ciruclar(X_train_observed_b, "sun_azimuth:d")
X_test_estimated_b = convert_from_degree_to_ciruclar(X_test_estimated_b, "sun_azimuth:d")
X_train_estimated_c = convert_from_degree_to_ciruclar(X_train_estimated_c, "sun_azimuth:d")
X_train_observed_c = convert_from_degree_to_ciruclar(X_train_observed_c, "sun_azimuth:d")
X_test_estimated_c = convert_from_degree_to_ciruclar(X_test_estimated_c, "sun_azimuth:d")

X_train_observed_a.head()

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,sun_azimuth:d_sin,sun_azimuth:d_cos
0,2019-06-02 22:00:00,7.7,1.230,1744.900024,0.0,0.0,1744.900024,0.0,280.299988,0.0,...,0.0,285.899994,100.0,39640.101562,3.7,-3.6,-0.8,-0.0,-0.295141,0.955454
1,2019-06-02 22:15:00,7.7,1.229,1734.000000,0.0,0.0,1734.000000,0.0,280.299988,0.0,...,0.0,286.100006,100.0,40123.898438,3.6,-3.6,-0.6,-0.0,-0.236940,0.971524
2,2019-06-02 22:30:00,7.7,1.228,1723.500000,0.0,0.0,1723.500000,0.0,280.299988,0.0,...,0.0,286.299988,100.0,40628.300781,3.6,-3.6,-0.4,-0.0,-0.177634,0.984097
3,2019-06-02 22:45:00,7.7,1.226,1713.400024,0.0,0.0,1713.400024,0.0,280.299988,0.0,...,0.0,286.600006,100.0,41153.601562,3.5,-3.5,-0.2,-0.0,-0.117520,0.993070
4,2019-06-02 23:00:00,7.7,1.225,1703.599976,0.0,0.0,1703.599976,0.0,280.299988,0.0,...,0.0,286.799988,100.0,41699.898438,3.5,-3.5,0.0,-0.0,-0.056832,0.998384


#### Converting every 4 quarters into an whole hour

In [342]:
## It might be better not to resample actually, but just use the actual hours that correspond perfectly

def convert_df_into_hourly(df):
    df.set_index("date_forecast", inplace=True)
    df = df.resample('1H').mean()
    df.reset_index(inplace=True)
    return df

In [343]:

X_train_estimated_a = convert_df_into_hourly(X_train_estimated_a)
X_train_observed_a = convert_df_into_hourly(X_train_observed_a)
X_test_estimated_a = convert_df_into_hourly(X_test_estimated_a)
X_train_estimated_b = convert_df_into_hourly(X_train_estimated_b)
X_train_observed_b = convert_df_into_hourly(X_train_observed_b)
X_test_estimated_b = convert_df_into_hourly(X_test_estimated_b)
X_train_estimated_c = convert_df_into_hourly(X_train_estimated_c)
X_train_observed_c = convert_df_into_hourly(X_train_observed_c)
X_test_estimated_c = convert_df_into_hourly(X_test_estimated_c)

## We should probably do it for the test_sets here as well

print("heu")


heu


#### Removing date_calc from all estimated sets


In [344]:
X_test_estimated_a.drop("date_calc", axis=1, inplace=True)
X_train_estimated_a.drop("date_calc", axis=1, inplace=True)
X_test_estimated_b.drop("date_calc", axis=1, inplace=True)
X_train_estimated_b.drop("date_calc", axis=1, inplace=True)
X_test_estimated_c.drop("date_calc", axis=1, inplace=True)
X_train_estimated_c.drop("date_calc", axis=1, inplace=True)

#### Removing all NaN from y_targets
 - Removing all NaN from y_targets and their corresponding dates in the X_train sets, both observed and estimated
 

In [345]:
#Function that returns train_targets, observed and estimated sets left after filtering away NaN
def drop_nan_rows_in_target_and_train(y_df, observed_train_df, estimated_train_df):
    y_df = y_df.dropna(subset=['pv_measurement'])
    valid_dates = y_df['time']
    observed_train_df = observed_train_df[observed_train_df['date_forecast'].isin(valid_dates)]
    estimated_train_df = estimated_train_df[estimated_train_df['date_forecast'].isin(valid_dates)]
    return (y_df, observed_train_df, estimated_train_df)

In [346]:
#doing the NaN filtering
y_a, X_train_observed_a, X_train_estimated_a = drop_nan_rows_in_target_and_train(y_a, X_train_observed_a, X_train_estimated_a)
y_b, X_train_observed_b, X_train_estimated_b = drop_nan_rows_in_target_and_train(y_b, X_train_observed_b, X_train_estimated_b)
y_c, X_train_observed_c, X_train_estimated_c = drop_nan_rows_in_target_and_train(y_c, X_train_observed_c, X_train_estimated_c)

y_c.head()

,time,pv_measurement
5913,2019-09-04 08:00:00,137.2
5914,2019-09-04 09:00:00,0.0
5915,2019-09-04 10:00:00,0.0
5916,2019-09-04 11:00:00,0.0
5917,2019-09-04 12:00:00,0.0


In [347]:
X_train_observed_c.head()

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,sun_azimuth:d_sin,sun_azimuth:d_cos
5912,2019-09-04 08:00:00,6.625,1.22075,2287.250000,1320844.375,421.225006,2287.250000,0.0,278.200012,67.974998,...,0.0,283.725006,31.199999,49549.449219,1.250,-0.825,0.825,0.0,0.759872,-0.645637
5913,2019-09-04 09:00:00,6.275,1.21425,2679.074951,1681730.500,508.125000,2679.074951,0.0,277.424988,76.625000,...,0.0,284.799988,86.250000,52309.976562,1.500,-1.500,-0.200,0.0,0.550184,-0.831061
5914,2019-09-04 10:00:00,5.900,1.20825,2983.750000,1936799.875,561.875000,2983.750000,0.0,276.650024,112.574997,...,0.0,285.899994,95.149994,53978.800781,2.000,-2.000,-0.225,0.0,0.279491,-0.956291
5915,2019-09-04 11:00:00,5.875,1.20350,3286.550049,2063526.250,578.025024,3286.550049,0.0,276.649994,195.149994,...,0.0,286.924988,100.000000,54510.925781,1.950,-1.950,-0.250,0.0,-0.027237,-0.995790
5916,2019-09-04 12:00:00,6.150,1.20175,3453.425049,2051320.000,555.174988,3453.425049,0.0,277.250000,243.375000,...,0.0,287.500000,100.000000,54284.648438,1.525,-1.500,-0.300,0.0,-0.330277,-0.940016


#### Making time features

In [348]:
#Function that creates timefeatures based on the specified coloumn
def create_time_features_based_on_coloun(df, col):   
    df['hour'] = df[col].dt.hour
    df['dayofmonth'] = df[col].dt.day
    df['dayofweek'] = df[col].dt.dayofweek
    df['quarter'] = df[col].dt.quarter
    df['month'] = df[col].dt.month
    df['year'] = df[col].dt.year
    df['dayofyear'] = df[col].dt.dayofyear
    return df


In [349]:
# adding time features

#A
X_train_estimated_a = create_time_features_based_on_coloun(X_train_estimated_a, "date_forecast")
X_train_observed_a = create_time_features_based_on_coloun(X_train_observed_a, "date_forecast")
X_test_estimated_a = create_time_features_based_on_coloun(X_test_estimated_a, "date_forecast")

#B
X_train_estimated_b = create_time_features_based_on_coloun(X_train_estimated_b, "date_forecast")
X_train_observed_b = create_time_features_based_on_coloun(X_train_observed_b, "date_forecast")
X_test_estimated_b = create_time_features_based_on_coloun(X_test_estimated_b, "date_forecast")

#C
X_train_estimated_c = create_time_features_based_on_coloun(X_train_estimated_c, "date_forecast")
X_train_observed_c = create_time_features_based_on_coloun(X_train_observed_c, "date_forecast")
X_test_estimated_c = create_time_features_based_on_coloun(X_test_estimated_c, "date_forecast")



Just showing the shapes to se if i havent f'ed up

In [350]:
# printing shapes
print("A")
print(X_train_estimated_a.shape)
print(X_train_observed_a.shape)
print(X_train_estimated_a.shape[0]+X_train_observed_a.shape[0])
print(y_a.shape)

print("B")
print(X_train_estimated_b.shape)
print(X_train_observed_b.shape)
print(X_train_estimated_b.shape[0]+X_train_observed_b.shape[0])  ## this one seems to be of by 1??????
print(y_b.shape)

print("C")
print(X_train_estimated_c.shape)
print(X_train_observed_c.shape)
print(X_train_estimated_c.shape[0]+X_train_observed_c.shape[0])
print(y_c.shape)

A
(4418, 53)
(29667, 53)
34085
(34085, 2)
B
(3625, 53)
(29218, 53)
32843
(32844, 2)
C
(2954, 53)
(23141, 53)
26095
(26095, 2)


We can see that for B there is actually on row more in the target set for some reason, finding that row and deleting it:

In [351]:
## Finding the row in y_b that does not have a match

# renaming
date_times_estimated = X_train_estimated_b['date_forecast']
date_times_observed = X_train_observed_b['date_forecast']
result_df = y_b[~y_b['time'].isin(date_times_estimated) & ~y_b['time'].isin(date_times_observed)]
print(result_df)


                 time  pv_measurement
0 2018-12-31 23:00:00             0.0


Okey so let's remove that row and verify that everything is correct:

In [352]:
# removing row
y_b = y_b[~y_b['time'].isin(result_df['time'])]

# checking that the numbers add up again
print("B")
print(X_train_estimated_b.shape)
print(X_train_observed_b.shape)
print(X_train_estimated_b.shape[0]+X_train_observed_b.shape[0])
print(y_b.shape)


B
(3625, 53)
(29218, 53)
32843
(32843, 2)


#### Removing repeated indices


#### Dividing the sets into training and validation

In [353]:
# first renaming the date_forecast columns to time  (DOES NOT WORK)
for df in list_of_all_estimated_and_observed_sets:               
    df.rename(columns={'date_forecast': 'time'}, inplace=True)

#maybe we can drop it here already actually
X_train_observed_a.head()



,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,wind_speed_w_1000hPa:ms,sun_azimuth:d_sin,sun_azimuth:d_cos,hour,dayofmonth,dayofweek,quarter,month,year,dayofyear
0,2019-06-02 22:00:00,7.700,1.22825,1728.949951,0.000000,0.00,1728.949951,0.0,280.299988,0.000,...,0.0,-0.206809,0.976036,22,2,6,2,6,2019,153
1,2019-06-02 23:00:00,7.700,1.22350,1689.824951,0.000000,0.00,1689.824951,0.0,280.299988,0.000,...,0.0,0.034484,0.997084,23,2,6,2,6,2019,153
2,2019-06-03 00:00:00,7.875,1.21975,1563.224976,0.000000,0.00,1563.224976,0.0,280.649994,0.000,...,0.0,0.257133,0.964021,0,3,0,2,6,2019,154
3,2019-06-03 01:00:00,8.425,1.21800,1283.425049,208.649994,0.75,1283.425049,0.0,281.674988,0.300,...,0.0,0.478111,0.875816,1,3,0,2,6,2019,154
4,2019-06-03 02:00:00,8.950,1.21800,1003.500000,32468.150391,23.10,1003.500000,0.0,282.500000,11.975,...,0.0,0.667714,0.741636,2,3,0,2,6,2019,154


Training data = first 75% of observed + first 75% of estimated
Validation data = last 25% of observed + last 25% of estimated

In [354]:



# Making training and validation data for A
percent_observed_train_a = 1
percent_estimated_train_a = 1

split_index_obs_a = int(len(X_train_observed_a)*percent_observed_train_a)
X_train_observed_a_first_75 = X_train_observed_a[:split_index_obs_a]
X_train_observed_a_last_25 = X_train_observed_a[split_index_obs_a:]

split_index_est_a = int(len(X_train_estimated_a)*percent_estimated_train_a)
X_train_estimated_a_first_75 = X_train_estimated_a[:split_index_est_a]
X_train_estimated_a_last_25 = X_train_estimated_a[split_index_est_a:]

X_train_a = pd.concat([X_train_observed_a_first_75, X_train_estimated_a_first_75])
y_train_a = y_a[y_a["time"].isin(X_train_a['date_forecast'])]
print(X_train_a.shape, y_train_a.shape)

X_validate_a = pd.concat([X_train_observed_a_last_25, X_train_estimated_a_last_25])
y_validate_a = y_a[y_a["time"].isin(X_validate_a['date_forecast'])]


# making training and validation for B
percent_observed_train_b = 1
percent_estimated_train_b = 1

split_index_obs_b = int(len(X_train_observed_b)*percent_observed_train_b)
X_train_observed_b_first_75 = X_train_observed_b[:split_index_obs_b]
X_train_observed_b_last_25 = X_train_observed_b[split_index_obs_b:]


split_index_est_b = int(len(X_train_estimated_b)*percent_estimated_train_b)
X_train_estimated_b_first_75 = X_train_estimated_b[:split_index_est_b]
X_train_estimated_b_last_25 = X_train_estimated_b[split_index_est_b:]

X_train_b = pd.concat([X_train_observed_b_first_75, X_train_estimated_b_first_75])
y_train_b = y_b[y_b["time"].isin(X_train_b['date_forecast'])]
print(X_train_b.shape, y_train_b.shape)

X_validate_b = pd.concat([X_train_observed_b_last_25, X_train_estimated_b_last_25])
y_validate_b = y_b[y_b["time"].isin(X_validate_b['date_forecast'])]

# making training and validation for C
percent_observed_train_c = 1
percent_estimated_train_c = 1

split_index_obs_c = int(len(X_train_observed_c)*percent_observed_train_c)
X_train_observed_c_first_75 = X_train_observed_c[:split_index_obs_c]
X_train_observed_c_last_25 = X_train_observed_c[split_index_obs_c:]

split_index_est_c = int(len(X_train_estimated_c)*percent_estimated_train_c)
X_train_estimated_c_first_75 = X_train_estimated_c[:split_index_est_c]
X_train_estimated_c_last_25 = X_train_estimated_c[split_index_est_c:]

X_train_c = pd.concat([X_train_observed_c_first_75, X_train_estimated_c_first_75])
y_train_c = y_c[y_c["time"].isin(X_train_c['date_forecast'])]
print(X_train_c.shape, y_train_c.shape)

X_validate_c = pd.concat([X_train_observed_c_last_25, X_train_estimated_c_last_25])
y_validate_c = y_c[y_c["time"].isin(X_validate_c['date_forecast'])]


(34085, 53) (34085, 2)
(32843, 53) (32843, 2)
(26095, 53) (26095, 2)


#### Removing repeated indexes here, think this might be the wrong place to do it, should probably to it for the concat BUT hEY

In [355]:

def find_repeated_indexes(df, column_name, repeat_count=12):
    """
    Find and return the indexes of rows with a specified number of repeated values in a given column.

    Parameters:
    - df: DataFrame to search for repeated rows.
    - column_name: Name of the column to check for repeated values.
    - repeat_count: Number of repeated values required to consider a row as a match.

    Returns:
    - List of indexes for rows with the specified number of repeated values in the given column.
    """
    df = df.reset_index()
    repeated_indexes = []
    temp_repeated_indexes = []
    current_value = None
    count = 0

    for index, row in df.iterrows():

        value = row[column_name]

        if value == current_value:
            count += 1
            temp_repeated_indexes.append(index)
        else:
            current_value = value
            if count <= repeat_count:
                temp_repeated_indexes = []
                count = 1
            else:
                for i in temp_repeated_indexes:
                    if i not in repeated_indexes:
                        repeated_indexes.append(i)
                temp_repeated_indexes = []
                count = 1
            

    return repeated_indexes



In [356]:
#removing repeated indexes

#A
print(X_train_a.shape, y_train_a.shape)
repeated_indices = find_repeated_indexes(y_train_a,"pv_measurement", 24)
y_train_a = y_train_a.reset_index()
y_train_a = y_train_a.drop(repeated_indices)
X_train_a = X_train_a[X_train_a["date_forecast"].isin(y_train_a["time"])]
print(X_train_a.shape, y_train_a.shape)

repeated_indices = find_repeated_indexes(y_validate_a,"pv_measurement",22)
y_validate_a = y_validate_a.reset_index()
y_validate_a = y_validate_a.drop(repeated_indices)
X_validate_a = X_validate_a[X_validate_a["date_forecast"].isin(y_validate_a["time"])]

y_train_a.reset_index(drop=True, inplace=True)
X_train_a.reset_index(drop=True, inplace=True)
y_validate_a.reset_index(drop=True, inplace=True)
X_validate_a.reset_index(drop=True, inplace=True)



(34085, 53) (34085, 2)


(34043, 53) (34043, 3)


In [357]:
#B  -  works reeaaaly well for B
print(X_train_b.shape, y_train_b.shape)
repeated_indices = find_repeated_indexes(y_train_b,"pv_measurement", 24)
y_train_b = y_train_b.reset_index()
y_train_b = y_train_b.drop(repeated_indices)
X_train_b = X_train_b[X_train_b["date_forecast"].isin(y_train_b["time"])]
print(X_train_b.shape, y_train_b.shape)

repeated_indices = find_repeated_indexes(y_validate_b,"pv_measurement",24)
y_validate_b = y_validate_b.reset_index()
y_validate_b = y_validate_b.drop(repeated_indices)
X_validate_b = X_validate_b[X_validate_b["date_forecast"].isin(y_validate_b["time"])]

y_train_b.reset_index(drop=True, inplace=True)
X_train_b.reset_index(drop=True, inplace=True)
y_validate_b.reset_index(drop=True, inplace=True)
X_validate_b.reset_index(drop=True, inplace=True)

(32843, 53) (32843, 2)
(25954, 53) (25954, 3)


In [358]:
#C
print(X_train_c.shape, y_train_c.shape)
repeated_indices = find_repeated_indexes(y_train_c,"pv_measurement", 24)
y_train_c = y_train_c.reset_index()
y_train_c = y_train_c.drop(repeated_indices)
X_train_c = X_train_c[X_train_c["date_forecast"].isin(y_train_c["time"])]
print(X_train_c.shape, y_train_c.shape)

repeated_indices = find_repeated_indexes(y_validate_c,"pv_measurement",24)
y_validate_c = y_validate_c.reset_index()
y_validate_c = y_validate_c.drop(repeated_indices)
X_validate_c = X_validate_c[X_validate_c["date_forecast"].isin(y_validate_c["time"])]

y_train_c.reset_index(drop=True, inplace=True)
X_train_c.reset_index(drop=True, inplace=True)
y_validate_c.reset_index(drop=True, inplace=True)
X_validate_c.reset_index(drop=True, inplace=True)

(26095, 53) (26095, 2)
(21169, 53) (21169, 3)


Removing time feature

In [359]:
X_train_a.drop("date_forecast", axis=1, inplace=True)
y_train_a.drop("time", axis=1, inplace=True)
X_validate_a.drop("date_forecast", axis=1, inplace=True)
y_validate_a.drop("time", axis=1, inplace=True)

X_train_b.drop("date_forecast", axis=1, inplace=True)
y_train_b.drop("time", axis=1, inplace=True)
X_validate_b.drop("date_forecast", axis=1, inplace=True)
y_validate_b.drop("time", axis=1, inplace=True)

X_train_c.drop("date_forecast", axis=1, inplace=True)
y_train_c.drop("time", axis=1, inplace=True)
X_validate_c.drop("date_forecast", axis=1, inplace=True)
y_validate_c.drop("time", axis=1, inplace=True)


Adding location features to the sets before merge

In [360]:
X_train_a["location"] =  "A" 
y_train_a["location"] = "A"
X_validate_a["location"] = "A"
y_validate_a["location"] = "A"

X_train_b["location"] = "B"
y_train_b["location"] = "B"
X_validate_b["location"] = "B"
y_validate_b["location"] = "B"

X_train_c["location"] = "C"
y_train_c["location"] = "C"
X_validate_c["location"] = "C"
y_validate_c["location"] = "C"


Merging tranining data:


In [361]:
X_train = pd.concat([X_train_a, X_train_b, X_train_c])
y_train = pd.concat([y_train_a, y_train_b, y_train_c])

X_validate = pd.concat([X_validate_a, X_validate_b, X_validate_c])
y_validate = pd.concat([y_validate_a, y_validate_b, y_validate_c])

print(X_train.shape)
print(y_train.shape)

(81166, 53)
(81166, 3)


One-hot encoding

In [362]:
# one hot encoding and dropping location feature   some fuckery here
# one_hot = pd.get_dummies(X_train["location"]).astype(int)
# X_train = X_train.drop("location", axis=1)
# X_train = pd.merge(X_train, one_hot, left_index=True, right_index=True)


# one_hot = pd.get_dummies(X_validate["location"]).astype(int)
# X_validate = pd.merge(X_validate, one_hot, left_index=True, right_index=True)


# print(X_train.shape)
# print(y_train.shape)

Fixing test set

In [363]:
# load valid dates function
import os


dir_path = os.getcwd()
def load_valid_dates():
    
    test = pd.read_csv(f"{dir_path}/../data/test.csv")

    return test["time"].unique().tolist()

In [364]:
#adding location feature

X_test_estimated_a["location"] = "A"
X_test_estimated_b["location"] = "B"
X_test_estimated_c["location"] = "C"

# concatting:
X_test = pd.concat([X_test_estimated_a, X_test_estimated_b, X_test_estimated_c])
# filtering out invalid dates:
X_test = X_test[X_test["date_forecast"].isin(load_valid_dates())]
# removing forecast coloum
X_test = X_test.drop("date_forecast", axis=1)

One-hot encoding:


In [365]:
X_train = X_train.reset_index().drop(columns="index")
one_hot = pd.get_dummies(X_train["location"]).astype(int)
X_train = X_train.drop("location", axis=1)
X_train = pd.merge(X_train, one_hot, left_index=True, right_index=True)

X_validate = X_validate.reset_index().drop(columns="index")
one_hot = pd.get_dummies(X_validate["location"]).astype(int)
X_validate = X_validate.drop("location", axis=1)
X_validate = pd.merge(X_validate, one_hot, left_index=True, right_index=True)

X_test = X_test.reset_index().drop(columns="index")
one_hot = pd.get_dummies(X_test["location"]).astype(int)
X_test = X_test.drop("location", axis=1)
X_test = pd.merge(X_test, one_hot, left_index=True, right_index=True)

X_test
# one_hot

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,hour,dayofmonth,dayofweek,quarter,month,year,dayofyear,A,B,C
0,4.325,1.28675,912.700012,0.000000,0.000000,1061.550049,0.0,271.650024,0.000000,0.000000,...,0,1,0,2,5,2023,121,1,0,0
1,4.275,1.28600,1482.099976,0.000000,0.000000,1075.100098,0.0,271.450012,0.000000,0.000000,...,1,1,0,2,5,2023,121,1,0,0
2,4.150,1.28375,1791.300049,0.000000,0.000000,1200.400024,0.0,271.049988,0.000000,0.000000,...,2,1,0,2,5,2023,121,1,0,0
3,4.025,1.28200,2312.875000,10124.424805,11.675000,1179.849976,0.0,270.649994,9.375000,16845.226562,...,3,1,0,2,5,2023,121,1,0,0
4,3.900,1.28100,2198.299805,141748.593750,76.875000,920.049988,0.0,270.375000,47.400002,102209.703125,...,4,1,0,2,5,2023,121,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,8.400,1.19675,3493.649902,475981.375000,84.775002,1967.375000,0.0,281.700012,32.200001,162567.531250,...,19,3,0,3,7,2023,184,0,0,1
2156,8.600,1.20000,3078.750000,183261.343750,24.549999,1449.500000,0.0,282.024994,13.875000,82875.351562,...,20,3,0,3,7,2023,184,0,0,1
2157,8.875,1.20350,2308.399902,36831.074219,1.225000,1543.650024,0.0,282.350006,1.250000,27210.250000,...,21,3,0,3,7,2023,184,0,0,1
2158,9.000,1.20650,2000.449951,344.575012,0.000000,1725.949951,0.0,282.600006,0.000000,2242.149902,...,22,3,0,3,7,2023,184,0,0,1


Normalizing data

In [366]:
X_train_mean = X_train.mean()
X_train_std = X_train.std()

X_train = (X_train - X_train_mean) / X_train_std
X_validate = (X_validate- X_train_mean) / X_train_std
X_test =  (X_test - X_train_mean) / X_train_std

X_test.head()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,hour,dayofmonth,dayofweek,quarter,month,year,dayofyear,A,B,C
0,-0.701343,0.935322,-0.785351,-0.650840,-0.649086,-0.383201,-0.06731,-0.627226,-0.670870,-0.678967,...,-1.665505,-1.685409,-1.501083,-0.435658,-0.418201,1.901447,-0.56653,1.176522,-0.685619,-0.593995
1,-0.720027,0.914424,-0.563534,-0.650840,-0.649086,-0.375777,-0.06731,-0.657592,-0.670870,-0.678967,...,-1.520728,-1.685409,-1.501083,-0.435658,-0.418201,1.901447,-0.56653,1.176522,-0.685619,-0.593995
2,-0.766735,0.851725,-0.443082,-0.650840,-0.649086,-0.307124,-0.06731,-0.718324,-0.670870,-0.678967,...,-1.375950,-1.685409,-1.501083,-0.435658,-0.418201,1.901447,-0.56653,1.176522,-0.685619,-0.593995
3,-0.813444,0.802954,-0.239895,-0.638766,-0.599089,-0.318384,-0.06731,-0.779052,-0.519252,-0.602391,...,-1.231173,-1.685409,-1.501083,-0.435658,-0.418201,1.901447,-0.56653,1.176522,-0.685619,-0.593995
4,-0.860152,0.775090,-0.284530,-0.481797,-0.319876,-0.460729,-0.06731,-0.820801,0.095707,-0.214333,...,-1.086395,-1.685409,-1.501083,-0.435658,-0.418201,1.901447,-0.56653,1.176522,-0.685619,-0.593995


In [367]:
# model = xgb.XGBRegressor(
#     max_depth=7,
#     colsample_bytree=0.8,
#     eta=0.1,
#     n_estimators=90,
#     reg_alpha=0.01,
#     reg_lambda=0.01,
#     enable_categorical=True
# )

# model.fit(X_train, y_train["pv_measurement"])

# # X_validate_a_loc = X_validate[X_validate_a["location"]=="A"]
# # X_validate_a_loc = X_validate_a_loc.drop("location", axis=1)
# prediction = model.predict(X_validate)
# # prediction_a = model.predict(X_validate_a)
# # prediction_b = model.predict(X_validate_b)
# # prediction_c = model.predict(X_validate_c)

In [368]:
# print(
#     "MAE: ",
#     round(
#         mean_absolute_error(
#             y_true=y_validate["pv_measurement"],
#             y_pred=prediction,
#         ),
#         3,
#     ),
# )

In [369]:
X_test.head()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,hour,dayofmonth,dayofweek,quarter,month,year,dayofyear,A,B,C
0,-0.701343,0.935322,-0.785351,-0.650840,-0.649086,-0.383201,-0.06731,-0.627226,-0.670870,-0.678967,...,-1.665505,-1.685409,-1.501083,-0.435658,-0.418201,1.901447,-0.56653,1.176522,-0.685619,-0.593995
1,-0.720027,0.914424,-0.563534,-0.650840,-0.649086,-0.375777,-0.06731,-0.657592,-0.670870,-0.678967,...,-1.520728,-1.685409,-1.501083,-0.435658,-0.418201,1.901447,-0.56653,1.176522,-0.685619,-0.593995
2,-0.766735,0.851725,-0.443082,-0.650840,-0.649086,-0.307124,-0.06731,-0.718324,-0.670870,-0.678967,...,-1.375950,-1.685409,-1.501083,-0.435658,-0.418201,1.901447,-0.56653,1.176522,-0.685619,-0.593995
3,-0.813444,0.802954,-0.239895,-0.638766,-0.599089,-0.318384,-0.06731,-0.779052,-0.519252,-0.602391,...,-1.231173,-1.685409,-1.501083,-0.435658,-0.418201,1.901447,-0.56653,1.176522,-0.685619,-0.593995
4,-0.860152,0.775090,-0.284530,-0.481797,-0.319876,-0.460729,-0.06731,-0.820801,0.095707,-0.214333,...,-1.086395,-1.685409,-1.501083,-0.435658,-0.418201,1.901447,-0.56653,1.176522,-0.685619,-0.593995


In [371]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

X_train = X_train.reset_index().drop(columns="index")
y_train = y_train.reset_index().drop(columns="index")
new_train = pd.merge(X_train, y_train["pv_measurement"], left_index=True, right_index=True)

X_validate = X_validate.reset_index().drop(columns="index")
y_validate = y_validate.reset_index().drop(columns="index")
new_validate = pd.merge(X_validate, y_validate["pv_measurement"], left_index=True, right_index=True)

h2o_train= h2o.H2OFrame(new_train)
h2o_validate= h2o.H2OFrame(new_validate)
h2o_test = h2o.H2OFrame(X_test)

aml = H2OAutoML(max_models=20, seed=1, stopping_metric="MAE", sort_metric="MAE", stopping_tolerance=0.5)
# aml.train(x=h2o_train.columns, y="pv_measurement", training_frame=h2o_train, validation_frame=h2o_validate)
aml.train(x=h2o_train.columns, y="pv_measurement", training_frame=h2o_train)
lb = aml.leaderboard
preds = aml.leader.predict(h2o_test)



Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,20 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.4
H2O_cluster_version_age:,8 days
H2O_cluster_name:,H2O_from_python_siver_w431g7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.468 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |█
08:03:24.413: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [372]:

lb.head(rows=lb.nrows)

model_id,mae,rmse,mse,rmsle,mean_residual_deviance
StackedEnsemble_BestOfFamily_1_AutoML_1_20231012_80324,87.7539,255.74,65402.7,nan,65402.7
XRT_1_AutoML_1_20231012_80324,88.6027,260.591,67907.7,0.611587,67907.7
DRF_1_AutoML_1_20231012_80324,88.8783,261.592,68430.6,0.61139,68430.6
StackedEnsemble_AllModels_1_AutoML_1_20231012_80324,90.7001,252.34,63675.3,nan,63675.3
GBM_grid_1_AutoML_1_20231012_80324_model_5,95.8858,261.336,68296.5,nan,68296.5
GBM_4_AutoML_1_20231012_80324,97.7373,264.427,69921.9,nan,69921.9
GBM_3_AutoML_1_20231012_80324,101.021,272.229,74108.8,nan,74108.8
GBM_grid_1_AutoML_1_20231012_80324_model_4,102.441,270.546,73195,nan,73195
GBM_1_AutoML_1_20231012_80324,102.564,277.042,76752.1,nan,76752.1
GBM_2_AutoML_1_20231012_80324,103.227,277.186,76832.3,nan,76832.3


In [373]:
preds.head()


predict
-3.89402
-3.89389
-3.73691
75.1189
392.88
815.678
1722.23
2753.58
2861.27
3129.58


In [374]:
leaderboard_df = lb.as_data_frame() 
prediction_df = preds.as_data_frame()

In [375]:
h2o.cluster().shutdown()

H2O session _sid_a33f closed.


In [376]:
prediction_df.head()

,predict
0,-3.894020
1,-3.893888
2,-3.736909
3,75.118880
4,392.880167


AUtoSKLearn

In [ ]:
# import autosklearn

In [83]:
# X_train = X_train.reset_index().drop(columns="index")
# y_train = y_train.reset_index().drop(columns="index")
# new_train = pd.merge(X_train, y_train["pv_measurement"], left_index=True, right_index=True)

# new_train.head()


,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,dayofmonth,dayofweek,quarter,month,year,dayofyear,A,B,C,pv_measurement
0,0.456297,-0.600671,-0.441131,-0.674940,-0.673148,-0.018225,-0.069583,0.592587,-0.690962,-0.699286,...,-1.570838,1.507596,-0.483891,-0.172822,-1.343939,-0.311773,1.147037,-0.691511,-0.569137,0.00
1,0.456297,-0.734137,-0.456531,-0.674940,-0.673148,-0.040015,-0.069583,0.592587,-0.690962,-0.699286,...,-1.570838,1.507596,-0.483891,-0.172822,-1.343939,-0.311773,1.147037,-0.691511,-0.569137,0.00
2,0.520623,-0.839508,-0.506364,-0.674940,-0.673148,-0.110523,-0.069583,0.645954,-0.690962,-0.699286,...,-1.456219,-1.497812,-0.483891,-0.172822,-1.343939,-0.301999,1.147037,-0.691511,-0.569137,0.00
3,0.722790,-0.888676,-0.616499,-0.674700,-0.670052,-0.266353,-0.069583,0.802239,-0.686243,-0.696957,...,-1.456219,-1.497812,-0.483891,-0.172822,-1.343939,-0.301999,1.147037,-0.691511,-0.569137,0.00
4,0.915767,-0.888676,-0.726684,-0.637615,-0.577787,-0.422253,-0.069583,0.928032,-0.502593,-0.601708,...,-1.456219,-1.497812,-0.483891,-0.172822,-1.343939,-0.301999,1.147037,-0.691511,-0.569137,19.36


In [84]:
# X_validate = X_validate.reset_index().drop(columns="index")
# y_validate = y_validate.reset_index().drop(columns="index")
# new_validate = pd.merge(X_validate, y_validate["pv_measurement"], left_index=True, right_index=True)

# new_validate.head()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,dayofmonth,dayofweek,quarter,month,year,dayofyear,A,B,C,pv_measurement
0,0.695222,-0.467205,-0.170594,-0.67494,-0.673148,-0.014884,4.415345,0.741250,-0.690962,-0.699286,...,1.409249,0.505793,1.33439,1.019317,1.552796,1.134735,1.147037,-0.691511,-0.569137,0.0
1,0.603328,-0.439109,0.466660,-0.67494,-0.673148,0.166078,4.415345,0.676450,-0.690962,-0.699286,...,1.409249,0.505793,1.33439,1.019317,1.552796,1.134735,1.147037,-0.691511,-0.569137,0.0
2,0.621706,-0.558524,-0.380887,-0.67494,-0.673148,-0.048244,4.415345,0.687883,-0.690962,-0.699286,...,1.523868,1.006694,1.33439,1.019317,1.552796,1.144509,1.147037,-0.691511,-0.569137,0.0
3,0.640085,-0.593647,-0.483071,-0.67494,-0.673148,-0.119253,4.415345,0.706942,-0.690962,-0.699286,...,1.523868,1.006694,1.33439,1.019317,1.552796,1.144509,1.147037,-0.691511,-0.569137,0.0
4,0.704411,-0.635795,-0.601207,-0.67494,-0.673148,-0.394281,4.415345,0.752683,-0.690962,-0.699286,...,1.523868,1.006694,1.33439,1.019317,1.552796,1.144509,1.147037,-0.691511,-0.569137,0.0


### Neural net

#### Okay lets create a submission

In [377]:
# predicting:
# predict = model.predict(X_test)
# print(predict)

resultframe = pd.DataFrame(columns = ["id", "prediction"])
resultframe["prediction"] = prediction_df
resultframe["id"] = range(len(resultframe))
resultframe.head()
# making the csv
# final = prediction.reset_index().reset_index().rename(columns={"level_0": "id", "y": "prediction"})[["id", "prediction"]]

resultframe.to_csv("submission_sivert_super_model_5.csv", index=False)